# Example notebook

Here we get the Article classes for all given bio.tools tools.

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from bh24_literature_mining.utils import load_biotools_pub

tools = load_biotools_pub("../biotoolspub/biotoolspub.tsv")

tools_lower = set()

for index, row in tools.iterrows():
    name = row["name"]
    biotoolsID = row["biotoolsID"]

    link = row["file_path"]
    # print(f"Name: {name}, PubMed ID: {biotoolsID}, Link: {link}")
    tools_lower.add(name.lower())


print(len(tools_lower))

15005


## Example on how to use europepmc_api

In [39]:
from pathlib import Path
from bh24_literature_mining.europepmc_api import EuropePMCClient, write_tool_mentions_to_file
from bh24_literature_mining.europepmc_api import identify_tool_mentions_in_sentences
import pandas as pd
client = EuropePMCClient()

results_list = []

for index, tool in tools.iterrows():
    if(index > 10):
        break
    # Call bio.tools query and get a list of Article objects
    tool_name = str(tool["name"]).lower()
    tool_id = str(tool["biotoolsID"])
    biotools_articles = client.search_mentions(tool_name)

    if len(biotools_articles) == 0:
        continue
    first_article = biotools_articles[0]

    relevant_parahraphs = client.get_relevant_paragraphs(first_article.pmcid, tool_name)
    if len(relevant_parahraphs) == 0:
        print("No relevant paragraphs found", tool_name)
        continue


    result = identify_tool_mentions_in_sentences(first_article.pmcid, tool_name, tool_id, relevant_parahraphs)
    print(len(result))
    results_list.extend(result)


No relevant paragraphs found subtiwiki
No relevant paragraphs found metapathways
No relevant paragraphs found metapathways
No relevant paragraphs found mzmine
No relevant paragraphs found iphop
1
No relevant paragraphs found jalview
1
1
1
1


In [32]:
results_list

[['PMC11491431',
  'Utilizing two methods for handling missing values—imputation or deletion—we created various cohorts: DM-all, DM-slim, OS-all, OS-slim, CSS-all, and CSS-slim.',
  [(112, 116, 'slim', 'SLiM_software')]],
 ['PMC11373567',
  'By contrast, ITHANET (https://www.ithanet.eu/; accessed on 17 April 2024) is a web portal initially conceived specifically for hemoglobinopathies, which, as of this writing, holds in its databases sequence and functional annotation for 3462 variants in 538 genes.',
  [(34, 41, 'ithanet', 'the_ithanet_portal')]],
 ['PMC11373567',
  'By contrast, ITHANET (https://www.ithanet.eu/; accessed on 17 April 2024) is a web portal initially conceived specifically for hemoglobinopathies, which, as of this writing, holds in its databases sequence and functional annotation for 3462 variants in 538 genes.',
  [(34, 41, 'ithanet', 'the_ithanet_portal')]],
 ['PMC11373567',
  'By contrast, ITHANET (https://www.ithanet.eu/; accessed on 17 April 2024) is a web portal 

In [40]:

result_df = pd.DataFrame(results_list, columns=["PMCID", "Sentence", "NER_Tags"])
result_df = result_df.explode("NER_Tags")

# p = Path(__file__).resolve().parents[2]
# print(p)
print("/home/tess/biohackathon2024/generate_data/ner_tag_final.csv")
result_df.to_csv("/home/tess/biohackathon2024/generate_data/ner_tag_final.csv", index=False)



/home/tess/biohackathon2024/generate_data/ner_tag_final.csv


In [35]:
len(results_list)

5

In [26]:
result_df.sample(1)

,PMCID,Sentence,NER_Tags
0,PMC11373567,"By contrast, ITHANET (https://www.ithanet.eu/;...","(34, 41, ithanet, the_ithanet_portal)"
